In [96]:
import re
import sys
import pprint
import json
from IPython.display import display
from colorama import Fore, Back, Style

In [97]:
with open('./results/lab1.json') as lab1_file:
    data = json.load(lab1_file)
tokens_chain = data['chain']
service_words = data['tables']['service_words']
operations = data['tables']['operations']
separators = data['tables']['separators']
constants = data['tables']['constants']
identifiers = data['tables']['identifiers']

In [133]:
nxtsymb = []
token_index = -1
wrapper()

['W', 0, 'program']
program
['I', 0, 'trivial']
['R', 4, ';']
end of program
['W', 1, 'var']
var
['I', 1, 'a']
['R', 3, ':']
['W', 3, 'integer']
var_type
['R', 4, ';']


In [103]:
def scan():
    global tokens_chain, token_index, nxtsymb
    token_index += 1
    if token_index < len(tokens_chain):
        nxtsymb = tokens_chain[token_index]
        print(nxtsymb)

In [130]:
def error(text):
    print(Fore.RED + 'Fatal:', Fore.BLACK + text, Style.RESET_ALL)
#     ahah/0

In [116]:
def check(response):
    if len(response):
        error(response)
        return False
    else:
        return True

In [118]:
def wrapper():
    global tokens_chain, token_index, nxtsymb
    scan()
    if nxtsymb[2] == 'program':
        check(program())
    print('end of program')
    scan()
    if nxtsymb[2] == 'var':
        check(var())
    

In [107]:
def program():
    global tokens_chain, token_index, nxtsymb
    print(Fore.GREEN + 'program' + Style.RESET_ALL)
    
    scan()
    if nxtsymb[0] != 'I':
        return 'expected identifier'
    scan()
    if nxtsymb[2] != ';':
        return 'expected ";"'
    return ''

In [132]:
def var():
    global tokens_chain, token_index, nxtsymb
    print(Fore.GREEN + 'var' + Style.RESET_ALL)
    
    scan()
    if nxtsymb[0] != 'I':
        return 'expexted identifier'
    scan()
    while nxtsymb[2] == ',':
        scan()
        if nxtsymb[0] != 'I':
            return 'expexted identifier'
        scan()
    if nxtsymb[2] != ':':
        return 'expected ":"'
    scan()
    check(var_type())
    scan()
    if nxtsymb[2] != ';':
        return 'expected ";"'
    return ''

In [123]:
def var_type():
    global tokens_chain, token_index, nxtsymb
    print(Fore.GREEN + 'var_type' + Style.RESET_ALL)
    
    if nxtsymb[2] in ['integer', 'real', 'string']:
        return ''
    elif nxtsymb[2] == 'array':
        scan()
        if nextsymb[2] != '[':
            return 'expected "["'
        scan()
        check(interval())
        scan()
        while nextsymb[2] == ',':
            scan()
            check(interval())
            scan()
        if nextsymb[2] != ']':
            return 'expected "]"'
        scan()
        if nextsymb[2] != 'of':
            return 'expected "of"'
        if nxtsymb[2] not in ['integer', 'real', 'string']:
            return 'expected variable type'
        return ''
    else:
        return 'expected variable type of array description'
        
            

In [124]:
def interval():
    global tokens_chain, token_index, nxtsymb
    print(Fore.GREEN + 'interval' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'C':
        return 'expected constant'
    scan()
    if nxtsymb[2] != '..':
        return 'expected ".."'
    scan()
    if nxtsymb[0] != 'C':
        return 'expected constant'
    return ''